In [1]:

import outlines
import json



In [102]:

from pydantic import BaseModel, Field
from typing import Optional, List, Annotated



class SubQuery(BaseModel):

    sub_query: Annotated[str, Field(description="原始问题的拆分的子问题，必须是陈述句，不要使用问句")]


class QueryDecomposition(BaseModel):
    thought: str = Field(description="回答原始问题的思考和推理过程，逐步列出解题思路，即一步一步解答原始问题的步骤")
    sub_queries: List[SubQuery] = Field(description="原始问题拆分成子问题列表")








In [34]:
print(QueryDecomposition.model_json_schema())

{'$defs': {'SubQuery': {'properties': {'sub_query': {'description': '原始问题的拆分的子问题，必须是陈述句，不要使用问句', 'title': 'Sub Query', 'type': 'string'}}, 'required': ['sub_query'], 'title': 'SubQuery', 'type': 'object'}}, 'properties': {'thought': {'description': '回答原始问题的思考和推理过程，包含一步一步解答原始问题的步骤', 'title': 'Thought', 'type': 'string'}, 'sub_queries': {'description': '原始问题拆分成子问题列表', 'items': {'$ref': '#/$defs/SubQuery'}, 'title': 'Sub Queries', 'type': 'array'}}, 'required': ['thought', 'sub_queries'], 'title': 'QueryDecomposition', 'type': 'object'}


In [35]:
from openai import OpenAI


openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)




In [103]:

def query_decomposition(query):

    system_prompt = """您是一位具备高级推理能力的专家AI助手。您的任务是为您的思考过程提供详细、逐步的解释，并拆分原始问题生成一系列有序的子问题。具体要求：
    
    ## 原始问题的思考和推理过程:thought
    - 思考原始问题，推理其中解答问题的步骤
    - 思考的过程要参考程序员设计数据库表格的思维，但不要随意猜测表格名称和内容
    - 思考过程不要做任何的假设
    - 思考要简洁、准确和真实
    - 仅针对关键信息进行思考和推理
    
    
    ## 子问题列表: sub_queries
    - 子问题要与原始问题紧密相连，尽量使用原始问题的描述
    - 子问题之间要尽量独立
    - 子问题列表的答案能够推理出原始问题的答案
    - 子问题是解答原始问题有序的步骤
    - 子问题要符合程序员的思维
    - 子问题尽量普通人理解的角度拆分
    - 子问题必须使用陈述的语句，不要使用问句或者疑问句
    - 子问题的主体要来自原始问题
    - 如果子问题是简单的问题，可以保持不变


    ## 输出格式
    - thought:原始问题的思考和推理过程，逐步列出解题思路
    - sub_queries: 原始问题拆分成子问题列表
    
    """

    system_prompt = """以下是一个数据库查询任务，请根据提供的问题进行如下操作：

    1. **问题分解**：将问题分解为若干有序的子问题，每个子问题应是独立且具体的，可以直接用于查询数据库的文本描述。  
    2. **子问题输出格式**：子问题按照逻辑顺序编号，并清晰地描述要解决的具体任务或查询目标。  
    3. **答案整合逻辑**：根据子问题的答案，推理出原始问题的答案，并在最终答案部分给出。  
    
    """
    
    
    chat_response = client.chat.completions.create(
        model="Qwen2.5-7B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ],
        temperature=0.001,
        # top_p=0.8,
        max_tokens=1024,
        extra_body={
            # "repetition_penalty": 1.05,
            "guided_json": QueryDecomposition.model_json_schema()
        },
    )
    # print("Chat response:", chat_response)
    # print(chat_response.choices[0].message.content)
    # print(chat_response)
    return chat_response.choices[0].message.content
    




In [104]:

query = "姚明和奥尼尔谁的身高高"
# query = "张三的父亲叫什么"
# query = "功率最大的发电机是什么时候购买的"
# query = "耗电量最大的设备是什么？为什么它耗电量这么大"
# query = "耗电量最大的设备名称是及原因"
# query= "张三购物费用比旅游费用多了多少"


response = query_decomposition(query)
print(response)


{
    "thought": "要回答这个问题，我们需要查询两人的身高数据，然后进行比较。",
    "sub_queries": [
        {
            "sub_query": "查询姚明的身高数据"
        },
        {
            "sub_query": "查询奥尼尔的身高数据"
        },
        {
            "sub_query": "比较姚明和奥尼尔的身高，确定谁更高"
        }
    ]
}


In [108]:


def judge_query_decomposition(query, sub_queries, thought):
    

    judge_system_prompt = """以下是一个原始问题及其子问题分解。请按照以下要求验证子问题分解的质量，检查标准：
    1. 检查每个子问题是否简洁，无不必要的扩充。
    2. 验证子问题是否逻辑正确，且能够解决原始问题。
    3. 如果子问题存在扩充或偏离原始问题，请根据与原始问题的相关性，根据需要精简、删除或改写子问题。

    根据原始问题和子问题列表，逐步思考，逐步推理新的子问题列表和思考。

    原始问题：
    - {query}
    
    思考和推理过程：
    - {thought}
    
    子问题列表：
    {sub_queries}

    """.format(**{"query": query, "sub_queries": sub_queries, "thought": thought})

    chat_response = client.chat.completions.create(
        model="Qwen2.5-7B-Instruct",
        messages=[
            {"role": "system", "content": "您是一个有益的人工智能助手。"},
            {"role": "user", "content": judge_system_prompt},
        ],
        temperature=0.001,
        # top_p=0.8,
        max_tokens=1024,
        extra_body={
            # "repetition_penalty": 1.05,
            "guided_json": QueryDecomposition.model_json_schema()
        },
    )
    # print("Chat response:", chat_response)
    # print(chat_response.choices[0].message.content)
    # print(chat_response)
    return chat_response.choices[0].message.content






In [113]:

# query = "姚明和奥尼尔谁的身高高"
# query = "张三的父亲叫什么"
# query = "功率最大的发电机是什么时候购买的"
# query = "耗电量最大的设备是什么？为什么它耗电量这么大"
# query = "耗电量最大的设备名称是及原因"
query= "张三购物费用比旅游费用多了多少"


response = query_decomposition(query)
print("---------------------原始query 分解 输出-------------------------------")
print(response)
response_dict = json.loads(response)

thought = response_dict["thought"]
sub_queries = json.dumps(response_dict["sub_queries"], ensure_ascii=False)

judge_sub_queries = judge_query_decomposition(query, sub_queries, thought)
print("---------------------子问题判断后的分解输出-------------------------------")
print(judge_sub_queries)



---------------------原始query 分解 输出-------------------------------
{
  "thought": "将问题分解为具体的子问题，以便从数据库中获取相关信息。",
  "sub_queries": [
    {
      "sub_query": "查询张三的购物费用总额是多少？"
    },
    {
      "sub_query": "查询张三的旅游费用总额是多少？"
    },
    {
      "sub_query": "计算张三购物费用总额与旅游费用总额的差值是多少？"
    }
  ]
}
---------------------子问题判断后的分解输出-------------------------------
{ "thought": "原始问题要求计算张三购物费用比旅游费用多出的金额。子问题列表中的每个子问题都直接针对原始问题的不同部分，且逻辑上是连贯的。每个子问题都是简洁的，没有不必要的扩充。因此，子问题列表是合理的。" 
, "sub_queries": [{"sub_query": "查询张三的购物费用总额是多少？"}, {"sub_query": "查询张三的旅游费用总额是多少？"}, {"sub_query": "计算张三购物费用总额与旅游费用总额的差值是多少？"}] }


In [175]:


"""

连续思维链的提示：即每次只生成一个子问题，然后根据当前子问题及子问题答案、原始问题，生成下一个问题
--- 这种方式仅通过Prompt控制不好，可能需要训练或者微调。

"""

from enum import Enum

class NextAction(str, Enum):
    continuing = "continuing"
    finished = "finished"



class NextSubQuery(BaseModel):

    sub_query: str = Field(description="当前或者下一步要生成的子问题")
    # next_action: NextAction = Field(description="子问题的答案能否回答查询，若不能，则为continuing；若能回答，则为finished")
    next_action: NextAction
    

def continue_cot_subquery(query):
    """
    - 子问题放入到<sub_query></sub_query>中
    - 反思当前问题是否已经可以根据子问题的答案回答，若不能回答，则返回"continuing"；若能回答查询，则返回"finished"。
    结果放入到<next_action></next_action>中
    """

    system_prompt = """你将要帮助将一个复杂的查询（query）逐步分解为多个子问题，每次仅生成一个简洁、完整、准确的子问题，并根据上一步的答案来生成后续的子问题。每个子问题的答案将成为下一步迭代的基础，直到最终能够完整解答原始query。
    ## 步骤:
    - 理解查询: 首先，认真分析并理解原始查询的问题内容，确保能够明确查询的目标。
    - 拆解问题: 将查询拆解成多个子问题。每个子问题应简洁、直接，避免过于复杂的问法。确保每个子问题只聚焦于查询中的一个关键点。
    - 迭代生成: 对于每一个子问题：
        - 生成子问题的答案，确保它简洁而清晰。
        - 基于答案继续生成下一个子问题。
        - 仅生成下一个子问题，不要输入其它无关的内容。
        - 判断下一个子问题是否继续生成（continuing）还是已完成(finished)
    
    
    ## 反思和优化
    - 在每次生成子问题时，要反思前一步是否清楚、准确地揭示了查询的关键点。避免冗余和不相关的问题。
    
    """
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    chat_response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",messages=messages, 
                                                   temperature=0,
                                                   # top_p=0.8,
                                                   max_tokens=1024,
                                                   extra_body={
                                                       "repetition_penalty": 1.05,
                                                       # "guide_json": NextSubQuery.model_json_schema(),
                                                       # "guided_decoding_backend": "outlines"
                                                   }
                                                  )

    
    
    return chat_response.choices[0].message.content
    


def gen_next_subqury(query, sub_query, sub_query_answer):

    system_prompt = """你将要帮助将一个复杂的查询（query）逐步分解为多个子问题，每次仅生成一个简洁、完整、准确的子问题，并根据上一步的答案思考和推理生成后续的子问题。
    ## 当前上下文:
    - 原始查询: {query}。 
    - 已分解过的子问题：{sub_queries}
    - 已分解过的子问题：{sub_query_answer}
   
    ## 反思和优化
    - 在每次生成子问题时，要反思前一步是否清楚、准确地揭示了查询的关键点。避免冗余和不相关的问题。

    ## **生成下一个子问题**
    - 如果当前答案是下一个子问题的前提，根据答案生成下一个子问题。新的问题应紧扣当前答案，进一步细化或推进原始查询的解决方案。
    - 确保问题简洁、准确，并推动任务向前发展。
    - 生成下一个新的子问题(next_sub_query)，不要输入其它无关的内容。
    - 生成后续下一个子问题的思考过程(thought)
    - 生成的后续子问题不包含已经分解过的子问题的内容
   """
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    chat_response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",messages=messages, 
                                                   temperature=0,
                                                   # top_p=0.8,
                                                   max_tokens=1024,
                                                   extra_body={
                                                       # "repetition_penalty": 1.05,
                                                       # "guide_json": NextSubQuery.model_json_schema(),
                                                       # "guided_decoding_backend": "outlines"
                                                   }
                                                  )

    
    
    return chat_response.choices[0].message.content





In [176]:
import random

query= "张三购物费用比旅游费用多了多少"


sub_query = continue_cot_subquery(query)
print("子问题： ",sub_query)
sub_query_answer = str(random.randint(10, 500)) + "元"
next_sub_query = gen_next_subqury(query, sub_query, sub_query_answer)

print(next_sub_query)



子问题：  张三的购物费用是多少？
thought: 首先需要明确张三的购物费用和旅游费用分别是多少。
next_sub_query: 张三的购物费用是多少？
